<a href="https://colab.research.google.com/github/frankausberlin/lazystudent/blob/main/nbs/00_Yamler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font size='+2' color='#005F6A'>**Yamler**</font>

> This class **scans** a **folder** and its **subfolders** for special **YAML**'s named by **ID** and provides an **organization** and **serialization** mechanism for the files.
---



The following **concepts** are implemented:
<table><tr><td><b>Concept</td><td><b>Implementation</td><td><b>Description</td></tr>
<tr><td>Root</td><td><code>y.root</td><td>Or working folder. This is the location of the root folder.</td></tr><tr></tr>
<tr><td>Yaml</td><td>naming<br><code>y.dump ()</td><td><code>&lt;id&gt;.yml</code> or <code>&lt;id&gt;_&lt;tag&gt;.yml</code> or <code>__&lt;config&gt;.yml</code><br>It only dump changed files - one dump for all</td></tr><tr></tr>
<tr><td>ID</td><td><code>y.id_len</td><td>Len of id string of the yaml files (<code>&lt;id&gt;.yml</code>)</td></tr><tr></tr>
<tr><td>Tag</td><td><code>y.tags</code><br><code>t.data[&lt;tag>]</td><td>List of all found tags (<code>&lt;id&gt;_&lt;tag&gt;.yml</code>)</td></tr><tr></tr>
<tr><td>Config</td><td><code>y.config</td><td>Configs for a topic starts with two '_' (<code>__&lt;config&gt;.yml</code>)</td></tr><tr></tr>
<tr><td>Topic</td><td><code>y.topic</td><td>Name of the current selected topic ('_' if root)</td></tr><tr></tr>
<tr><td></td><td><code>y.folder</td><td>The topic folder is a subfolder in the root directory</td></tr><tr></tr>
<tr><td></td><td><code>y.data</td><td>All id associated yaml files, accessible via its tag (tag '_' for none tag)<br>e.g. <code>y.data['en']</cod> for <code>&lt;id&gt;_en.yml</code> or y.data['_']</code> for <code>&lt;id&gt;.yml</code></code></td></tr><tr></tr>
<tr><td></td><td><code>y.topics</td><td>All topics in a dict with topic name as key ('_' for root topic/folder)</td></tr><tr></tr>
<tr><td>Switching</td><td><code>y.activate (topic)</td><td>Set a topic (topic-name, folder, data, config) to the curent.</td></tr><tr></tr>
<tr><td>Access</td><td><code>y.data[&lt;tag>]<br>y.config[&lt;config>]</td><td>Manipulating data - sample:<br><li>  add a dict calles 'msg' to the file __collection.yml in current topic<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<code>y.config['collection']['msg'] = {'txt':'hello world'}</code><br><li> add a dict calles 'msg' to the file '8SF_h3xF3cE.yml'<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<code>y.data['8SF_h3xF3cE']['lst'] = ['a','b']</td></tr><tr></tr>
<tr><td></td><td></td><td></td></tr><tr></tr></table>

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

import os, yaml
from pathlib import Path

%cd ~/labor/gits/lazystudent/

/home/frank/labor/gits/lazystudent


In [ ]:
#| export

class Yamler:
  def __init__(self, root='yamler', id_len=11, onInit={'config':False, 'files':False, 'data':False}, mediator=None):
    """ set workingfolder, scan for yamls, load main yamls and build data object
    """
    # init attributes
    self.tags, self.id_len, self.topics, self.original, self.onInit, self.mediator = [],  id_len, {}, {}, onInit, mediator
    # init current working topic - can be root ('_') or an other topic from self.topics
    self.data, self.folder, self.files, self.topic, self.config = None, None, None, '_', None

    # replace ~, create root folder if not exists
    folder = os.path.expanduser(root)
    if not os.path.exists (folder): os.makedirs (root)

    # create Yamler root dict with empty file list, folder and empty data / config objects
    self.topics['_'] = {'files':[], 'folder':root, 'data': {}, 'config': {}}
    #self.root ={'files':[], 'folder':folder, 'data': {}, 'config': {}}

    # set root to working folder
    self.switch()

    # load yamls like in onInit
    self.initialize ()

    # use the subfolders as topics
    for item in os.listdir(f"{self.folder}/"):
      if os.path.isdir(os.path.join(f"{self.topics['_']['folder']}/", item)):

        # create topic without file list
        self.topics[item] = {'files':[], 'data': {}, 'config': {}, 'folder': f"{self.topics['_']['folder']}/{item}"}

        # load yamls like in onInit
        self.initialize (topic=item)

    # set root to working folder back - it changed by initializing topics
    self.switch()

    # say ready
    if self.mediator: self.mediator.notify ("onYamlerReady", list(self.topics.keys()))

  def activate (self, topic=None):
    self.switch (topic)
    self.scan ()
    if self.mediator: self.mediator.notify ("onYamlerActivate", self.topic)

  def initialize (self, topic=None):
    if topic: self.switch (topic)

    # build file list
    if self.onInit['files']: self.scan ('files')

    # config data ('__')
    if self.onInit['config']: self.scan ('config')

    # load all yamls for id
    if self.onInit['data']: self.scan ('data')

  def switch (self, topic=None):
    if not topic: topic = '_'
    # set topic to current
    self.topic, self.data,                  self.config,                  self.folder,                  self.files =\
    topic,      self.topics[topic]['data'], self.topics[topic]['config'], self.topics[topic]['folder'], self.topics[topic]['files']


  def scan (self, part='all'):
    if (part == 'files' or part == 'all') and not self.files:
      self.files = [f for f in os.listdir(f"{self.folder}") if f.split('.')[-1] == 'yml']

    if (part == 'config' or part == 'all') and not self.config:
      files = self.files if self.files else [f for f in os.listdir(f"{self.folder}") if f.split('.')[-1] == 'yml']
      for f in files:
        if f[:2] == '__' and not f[2:-4] in self.config:
          with open (f"{self.folder}/{f}", 'r') as stream:
            self.config[f[2:-4]] = yaml.safe_load(stream)
          self.original[f"{self.folder}/{f}"] = Yamler.dictcopy (self.config[f[2:-4]])

    if (part == 'data' or part == 'all') and not self.data:
      self.data = {}
      for id in self.ids():
        # only if not done before
        if id in self.data: continue # already loaded

        # main-yaml
        self.data[id] = {'_':{}} # empty data obj for id
        with open (f"{self.folder}/{id}.yml", 'r') as stream:
          self.data[id]['_'] = yaml.safe_load(stream)
        self.original[f"{self.folder}/{id}.yml"] = Yamler.dictcopy (self.data[id]['_'])

        # tag-yamls
        taglist = [f[self.id_len+1:-4] for f in self.files if len(f) > self.id_len+5 and f[self.id_len] == '_']
        for tag in taglist:
          # remember found tags
          if tag not in self.tags: self.tags.append(tag)
          try: # load if exists
            with open (f"{self.folder}/{id}_{tag}.yml", 'r') as stream: self.data[id][tag] = yaml.safe_load(stream)
            self.original[f"{self.folder}/{id}_{tag}.yml"] = Yamler.dictcopy (self.data[id][tag])
          except: pass

  def dumpChanged (self,fname,obj):
    count = 0
    # only if new or changed
    if not fname in self.original or self.original[fname] != obj:
      # write it
      with open(fname, 'w') as file: yaml.dump(obj, file)
      count += 1
      # delete old original
      if fname in self.original: del self.original[fname]
      # make a copy to compare if changed by next dump
      self.original[fname] = Yamler.dictcopy (obj)
    return count

  def dump (self):
    count = 0
    # write __<config>.yml's
    for config_yml in self.config:
      fname, obj = f"{self.folder}/__{config_yml}.yml", self.config[config_yml]
      count += self.dumpChanged (fname, obj)

    # write <id>.yml's
    for id in self.data:

      # main-yaml
      if not '_' in self.data[id]: continue
      fname, obj = f"{self.folder}/{id}.yml", self.data[id]['_']
      count += self.dumpChanged (fname, obj)

      # tag-yamls
      for tag in self.tags:
        if not tag in self.data[id]: continue
        fname, obj = f"{self.folder}/{id}_{tag}.yml", self.data[id][tag]
        count += self.dumpChanged (fname, obj)
    return count

  def ids(self):
    # get files
    files = self.files if self.files else [f for f in os.listdir(f"{self.folder}") if f.split('.')[-1] == 'yml']
    # parse ids - criterias: fname don't start with '__', correkt len and its a .yml-file
    return list(set([f[:-4] for f in files if f[:2] != '__' and len (f) == (self.id_len+4) and f[-4:] == '.yml']))

  def dictcopy (d):
    ret = {}
    if type(d) == dict: # data is dict
      ret = {}
      for k in d: # keys
        if type(d[k]) == dict: ret[k] = Yamler.dictcopy(d[k])
        if type(d[k]) == list: ret[k] = Yamler.dictcopy(d[k])
        else:                  ret[k] = d[k]
    elif type(d) == list: # data is list
      ret = []
      for i in d: # items
        if type(i) == dict: ret.append (Yamler.dictcopy(i))
        if type(i) == list: ret.append (Yamler.dictcopy(i))
        else:               ret.append (i)
    return ret

  def remove (self, id, deleteFiles=True):
    if deleteFiles:     os.system (f'rm {self.folder}/{id}*.yml')
    if id in self.data: del self.data[id]

  def store(self, id, data=None):
    pass






In [ ]:
#big test
!rm -rf yemler/; mkdir yemler; mkdir yemler/mathpl
!echo "test: 'hallo'" > yemler/01234567890.yml
!echo "test2: 'hallo2'" > yemler/01234567890_loop.yml
!echo "test3: 'hallo3'" > yemler/__info.yml
!echo "test4: 'hallo4'" > yemler/01234567890_de.yml
!echo "test: 'hallo'" > yemler/11234567890.yml
!echo "test2: 'hallo2'" > yemler/11234567890_loop.yml

!echo "test: 'hallo'" > yemler/mathpl/x1234567890.yml
!echo "test2: 'hallo2'" > yemler/mathpl/x1234567890_loop.yml
!echo "test3: 'hallo3'" > yemler/mathpl/__readme.yml
!echo "test4: 'hallo4'" > yemler/mathpl/x1234567890_de.yml
!echo "test: 'hallo'" > yemler/mathpl/x2234567890.yml
!echo "test2: 'hallo2'" > yemler/mathpl/x2234567890_loop.yml

y = Yamler (root='yemler')
y.activate()
# conf
y.data['01234567890']['_']['test'] += ' welta'
y.data['01234567890']['loop']['test2'] += ' weltb'
y.data['01234567890']['de']['test4'] += ' weltc'
y.config['info']['test3'] += ' weltd'
y.data['11234567890']['_']['test'] += ' weltx'
y.data['11234567890']['loop']['test2'] += ' weltb'
y.dump ()

y.activate ('mathpl')
y.data['x1234567890']['loop']['test2'] += ' weltx'
y.data['x1234567890']['de']['test4'] += ' weltx'
y.data['x1234567890']['_']['test'] += ' weltx'
y.data['x2234567890']['_']['test'] += ' weltx'
y.config['readme']['test3'] += ' weltx'
y.dump ()


print ('root:')
!cat yemler/01234567890.yml yemler/01234567890_loop.yml yemler/__conf.yml yemler/01234567890_de.yml yemler/11234567890.yml yemler/11234567890_loop.yml
print ('topic matpl:')
!cat yemler/mathpl/x1234567890.yml yemler/mathpl/x1234567890_loop.yml yemler/mathpl/__cunf.yml yemler/mathpl/x1234567890_de.yml yemler/mathpl/x2234567890.yml yemler/mathpl/x2234567890_loop.yml


root:
test: hallo welta
test2: hallo2 weltb
cat: yemler/__conf.yml: No such file or directory
test4: hallo4 weltc
test: hallo weltx
test2: hallo2 weltb
topic matpl:
test: hallo weltx
test2: hallo2 weltx
cat: yemler/mathpl/__cunf.yml: No such file or directory
test4: hallo4 weltx
test: hallo weltx
test2: 'hallo2'


In [ ]:
#| hide
%cd ~/labor/gits/lazystudent/
import nbdev; nbdev.nbdev_export()

/home/frank/labor/gits/lazystudent
